In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


In [0]:
pwd

'/content'

In [0]:
import os
total_len = 0
for i in os.listdir('/content/drive/My Drive/VipAssign/train'):
  total_len += len(os.listdir('/content/drive/My Drive/VipAssign/train/'+i))

print(total_len)

6627


In [0]:
ls "/content/drive/My Drive/VipAssign/test"

beach/            desert/    forest/           island/  mountain/     river/
commercial_area/  farmland/  industrial_area/  lake/    residential/  wetland/


In [0]:
ls 'bottleneck_fc_model.h5'

ls: cannot access 'bottleneck_fc_model.h5': No such file or directory


In [0]:
# save_bottlebeck feature
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

#solve keras is not defined issue
from keras import utils as np_utils
import keras

# dimensions of our images.
img_width, img_height = 256, 256

top_model_weights_path = "/content/drive/My Drive/VipAssign/bottleneck_fc_model.h5"
train_data_dir         = "/content/drive/My Drive/VipAssign/train"
validation_data_dir    = "/content/drive/My Drive/VipAssign/test"

nb_train_samples = 6600
nb_validation_samples = 1800
epochs = 50
batch_size = 20


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        #(own)-change from None to Categorical
        class_mode='categorical',
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save('bottleneck_features_validation.npy',bottleneck_features_validation)
    nb_validation_labels=generator.classes

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        #(own)-change from None to Categorical
        class_mode='categorical',
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save('bottleneck_features_train.npy',bottleneck_features_train)    
    nb_train_labels=generator.classes
    
    
   
    
    
    return(nb_validation_labels,nb_train_labels)

 
  
  
nb_validation_labels,nb_train_labels=save_bottlebeck_features()


Using TensorFlow backend.






58892288/58889256 [==============================] - 1s 0us/step


Found 1800 images belonging to 12 classes.
Found 6627 images belonging to 12 classes.


In [0]:
print(nb_train_samples)
print(nb_train_labels.size)
print(nb_validation_samples)
print(nb_validation_labels)

6600
6627
1800
[ 0  0  0 ... 11 11 11]


In [0]:

def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')
    #train_labels = np.array([0] * (nb_train_samples // 8) + [1] * (nb_train_samples // 8)+[2] * (nb_train_samples // 8) + [3] * (nb_train_samples // 8)+[4] * (nb_train_samples // 8) + [5] * (nb_train_samples // 8)+[6] * (nb_train_samples // 8) + [7] * (nb_train_samples // 8))
    #(own)
    num_classes=12

    
    train_labels = keras.utils.to_categorical(y=nb_train_labels, num_classes=num_classes)
    
#     labels[[0,0,1,0],
#            [0,0,0,1],
#            [0,1,0,0]]
    
    print(train_labels)
    
    validation_data = np.load('bottleneck_features_validation.npy')
    #validation_labels = np.array([0] * (nb_validation_samples // 8) + [1] * (nb_validation_samples // 8)+[2] * (nb_validation_samples // 8) + [3] * (nb_validation_samples // 8)+[4] * (nb_validation_samples // 8) + [5] * (nb_validation_samples // 8)+[6] * (nb_validation_samples // 8) + [7] * (nb_validation_samples // 8))
    #(own)
    validation_labels = keras.utils.to_categorical(nb_validation_labels, num_classes)
    
    #can use scikit learn to build the layers
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    #(own)8,softmax
    model.add(Dense(12, activation='softmax'))
    #categorical
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
    
    
train_top_model()

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



ValueError: ignored

In [0]:
#fine-tuning

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'

# dimensions of our images.
img_width, img_height = 256,256

train_data_dir="/content/drive/My Drive/VipAssign/train"
validation_data_dir="/content/drive/My Drive/VipAssign/test"
nb_train_samples =6600
nb_validation_samples = 1800
epochs = 50
batch_size = 20

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(8, activation='softmax'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
# model.add(top_model)
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

model.summary()

# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose=2)